In [2]:
import matplotlib.pyplot as plt
import numpy as np
import ctypes as ctypes

lib_path = "./../../../../mylib/target/debug/mylib.dll"
my_lib = ctypes.cdll.LoadLibrary(lib_path)

## --------------------------init_linear_model--------------------------
# pub extern "C" fn init_linear_model(input_count: u32) -> *mut LinearModel;
my_lib.init_linear_model.argtypes = [ctypes.c_uint32]
my_lib.init_linear_model.restype = ctypes.c_void_p

# pub extern "C" fn train_linear_model(model: *mut LinearModel, features: *const c_float, labels: *const c_float, data_size: u32, learning_rate: f32, epochs: u32)
my_lib.train_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float),
                                      ctypes.c_uint32, ctypes.c_float, ctypes.c_uint32]
my_lib.train_linear_model.restype = None

# pub extern "C" fn predict_linear_model(model: *mut LinearModel, inputs: *mut f32) -> c_float
my_lib.predict_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
my_lib.predict_linear_model.restype = ctypes.c_float

# pub extern "C" fn free_linear_model(model: *mut LinearModel)
my_lib.free_linear_model.argtypes = [ctypes.c_void_p]
my_lib.free_linear_model.restype = None




In [3]:
X = np.array([
    [1, 1],
    [2, 3],
    [3, 3]
])
Y = np.array([
    1,
    -1,
    -1
])

X_flat = X.flatten().astype(ctypes.c_float)
Y_flat = Y.flatten().astype(ctypes.c_float)
X_flat_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_flat_ptr = Y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
data_size = len(Y)

linear_model = my_lib.init_linear_model(2)
my_lib.train_linear_model(linear_model, X_flat_ptr, Y_flat_ptr, data_size, 0.001, 100000)



In [4]:
background_points = []
background_colors = []
x1 = 0

while x1 < 3:
    x2 = 0
    while x2 < 3:
        background_points.append([x1, x2])
        x2 = x2 + 0.1
    x1 += x1 + 0.1


In [ ]:
for point in background_points:
    points_pointer = np.ctypeslib.as_ctypes(np.array(point, dtype=ctypes.c_float))
    #weighted_sum = my_lib.predict_linear_model(linear_model, points_pointer)
    #print(weighted_sum)
    if my_lib.predict_linear_model(linear_model, points_pointer) >= 0:
        background_colors.append('lightblue')
    else:
        background_colors.append('pink')

print(background_colors)

In [ ]:
background_points = np.array(background_points)
plt.scatter(background_points[:, 0], background_points[:, 1], c=background_colors)
plt.scatter(X[0, 0], X[0, 1], color='blue')
plt.scatter(X[1:3, 0], X[1:3, 1], color='red')

plt.plot()
plt.show()
plt.clf()

my_lib.free_linear_model(linear_model)